In [1]:
!which python

/home/nico/Documents/Ing3/data-battle-2025/.venv/bin/python


In [2]:
import pymupdf
import ast

In [29]:
def crop_header_footer_pdf_page(page):
    height, width = page.rect.height, page.rect.width
    header = min(0.06 * height, height)  # Ensure header is within bounds
    footer = min(0.06 * height, height)  # Ensure footer is within bounds

    # Ensure the crop box is within the media box
    crop_box = (0, header, width , height - footer)

    if crop_box[1] < crop_box[3] and crop_box[3] <= height:
        page.set_cropbox(crop_box)
    else:
        raise ValueError("Calculated crop box is outside the media box boundaries.")
    return page



def extract_question_block(lines):
    """
    extract each question block from the list of lines
    each question start from a line with 'Question X' in blod font
    and ends with a line with 'Question X+1' in bold font
    
    X goes from 1 to 20
    The question number 20 ends with the last line of the last page
    
    
    :param lines: list of lines, each line is a list of spans
    """
    questions = []
    question = []
    question_number = 1

    for line in lines:
        if 'bold' in line['font'].lower():
            if line['text'].lower().startswith(f'question {question_number}'):

                if question:
                    questions.append(question.copy())
                    question = []
                question_number += 1
            
        question.append(line.copy())

    questions.append(question.copy())
    return questions



def build_line(spans):
    output = spans[0].copy()

    for span in spans[1::]:
        output['text'] += span['text']

    return output


In [ ]:

def build_line(spans):
    output = spans[0].copy()

    fonts = {
    }

    fonts[spans[0]['font']] = len(spans[0]['text'].replace(' ', ''))
    
    for span in spans[1::]:
        output['text'] += span['text']

        fonts[span['font']] = fonts.get(span['font'], 0) + len(span['text'].replace(' ', ''))

    output['font'] = max(fonts, key=fonts.get)
    if len(output['text'].replace(' ', '')) <= 5:
        output['font'] = 'arial'

    return output

    
def extract_bold_block(lines):
    """
    extract each bold block from the list of lines
    each block start from a line in blod font
    and ends with a line in bold font
    
    
    :param lines: list of lines, each line is a list of spans
    """
    bold_blocks = []
    bold_block = []

    for line in lines:
        if 'bold' in line['font'].lower():
            if bold_block:
                bold_blocks.append(bold_block.copy())
                bold_block = []
            
        bold_block.append(line.copy())

    bold_blocks.append(bold_block.copy())
    return bold_blocks



# ----------------- MAIN -----------------

pdf = pymupdf.open('../data/EQE/2012_PreEx_answers_EN.pdf')
pdf = pymupdf.open('../data/EQE/2015_PreEx_questions_en.pdf')
pdf = pymupdf.open('../data/EQE/2014_PreEx_Questions.pdf')
pdf = pymupdf.open('../data/EQE/2015_PreEx_answers.pdf')
# pdf = pymupdf.open('../data/EQE/2014_PreEx_answers.pdf')

lines = []
for page in pdf[:-1]:
    page = crop_header_footer_pdf_page(page)
    
    
    for block in page.get_text('dict', flags=11)['blocks']:
        for line in block['lines']:
            # try:
            lines.append(build_line(line['spans'].copy()))
            # except:
            #     print(line["spans"])
            #     print("\n\n\n\n")
            #     exit(1)
    
pdf.save('../data/EQE/output.pdf')


# count how many lines are in bold
bold_count = 0
for line in lines:
    if 'bold' in line['font'].lower():
        # print(line['text'] + '\n\n')
        bold_count += 1
    
print(f"Number of bold lines: {bold_count}")


questions = extract_bold_block(lines)
print("bold BLOCKS FIND")
print(len(questions))
print("\n\n\n\n\n")

Q1_10 = []
q_id = 1
for question in questions[::]:

    unique_question = []
    for line in question:
        if line in unique_question:
            continue
        unique_question.append(line)


    # print(unique_question[0]['text'])
    if f'question {q_id}' in unique_question[0]['text'].lower():
        Q1_10.append(unique_question)
        q_id += 1

    # for line in unique_question:
    # for span in unique_question:
    #     # for span in line['spans']:
    #     print(f"Font: {span['font']}\nText: {span['text']}\nSize: {span['size']}\nBbox: {span['bbox']}\n-------------------")
    #     print('\t ---------- EOL ----------')
    # print('\t ---------- EOBlock ----------\n\n\n')

print(len(Q1_10[:10]))

for q in Q1_10[:10]:
    


    




_IncompleteInputError: incomplete input (3782885534.py, line 117)

In [ ]:

    # extract all data & xref of the images
    for image_idx, img in enumerate(image_list, start=1):
        xref = img[0]
        
        base_image = pdf_file.extract_image(xref)

        image_bytes = base_image["image"]
        image_ext = base_image["ext"]
        image_list_data.append((xref, image_bytes, image_ext))

print(f"Number of images before merging: {len(image_list_data)}")
print(image_list_data[1][1])

# merge continuous images with the same xref
image_list = merge_continuous_xref_images(image_list_data)
print(f"Number of images after merging: {len(image_list)}")
for image_index, img in enumerate(image_list, start=1):
    
    image_name = f"image{image_index}.{img[2]}"
    with open(image_name, "wb") as image_file:
        image_file.write(img[1])
        print(f"[+] Image saved as {image_name}")



In [ ]:
def merge_continuous_xref_images(image_list_data):
    """
    merge continuous images with the same xref

    Args:
        image_list_data: list of tuples (xref, image_bytes, image_ext)
    """
    merged_images = []
    current_xref = -1
    current_image = None

    for xref, image_bytes, image_ext in image_list_data:
        if xref == current_xref + 1:
            current_image += image_bytes
            current_xref = xref

        else:
            if current_xref != -1:
                merged_images.append((current_xref, current_image, image_ext))
            current_xref = xref
            current_image = image_bytes

    if current_xref:
        merged_images.append((current_xref, current_image, image_ext))

    return merged_images


pdf_file = pdf
image_list_data = []
for page_index in range(len(pdf_file)):

    # get the page itself
    page = pdf_file.load_page(page_index)  # load the page
    image_list = page.get_images(full=True)  # get images on the page

    # printing number of images found in this page
    if image_list:
        print(f"[+] Found a total of {len(image_list)} images on page {page_index}")
    else:
        print("[!] No images found on page", page_index)
    

    # merge all images of the same page one under the other
    for image_index, img in enumerate(image_list, start=1):
        # get the XREF of the image
        xref = img[0]
        print(f"Image {image_index} XREF: {xref}")

        # extract the image bytes
        base_image = pdf_file.extract_image(xref)
        image_bytes = base_image["image"]

        # get the image extension
        image_ext = base_image["ext"]

        # save the image
        image_name = f"image{page_index+1}_{image_index}.{image_ext}"
        with open(image_name, "wb") as image_file:
            image_file.write(image_bytes)
            print(f"[+] Image saved as {image_name}")


Number of images before merging: 94
b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x11\x1d\x00\x00\x00\xef\x08\x00\x00\x00\x00D\xda\x9b\xca\x00\x00\x00\tpHYs\x00\x00\x0e\xc4\x00\x00\x0e\xc4\x01\x95+\x0e\x1b\x00\x00\x1fyIDATx\x9c\xed\xdd\xedv\xe46\xae\x05\xd0\xbc\xffK\xfb\xae\x99\xb9\xe9\xb6]\xfa A\x80\x12\xa5\xbd\xffM\'\x10Y\x16\x8b@\x9fI\xd2\xff\xfc\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xbc\xd6\x17<\xcc\xd5\xdf)\x00\x8c\x17<\xcd\xd5_)\x00\xea]\xddk \xdb\xd5\xdf)\x00\x8c\x17<\xce\xd5\xdf)\x00\xca]\xddj \xdb\xd5\xdf)\x00\x

In [49]:


pdf = pymupdf.open('../data/EQE/2015_PreEx_questions_en.pdf')
pdf = pymupdf.open('../data/EQE/2014_PreEx_Questions.pdf')
pdf = pymupdf.open('../data/EQE/2015_PreEx_answers.pdf')
pdf = pymupdf.open('../data/EQE/2012_PreEx_answers_EN.pdf')
pdf = pymupdf.open('../data/EQE/2017_PreEx_answers.pdf')
# pdf = pymupdf.open('../data/EQE/2014_PreEx_answers.pdf')

lines = []
for page in pdf[:-1]:
    page = crop_header_footer_pdf_page(page)
    
    
    for block in page.get_text('dict', flags=11)['blocks']:
        for line in block['lines']:
            # try:
            lines.append(build_line(line['spans'].copy()))
            # except:
            #     print(line["spans"])
            #     print("\n\n\n\n")
            #     exit(1)
    
pdf.save('../data/EQE/output.pdf')

   

# print(lines)
print("PRETRAITEMENT")
# for line in lines:
#     print(f"Font: {line['font']}\nText: {line['text']}\nSize: {line['size']}\nBbox: {line['bbox']}\n-------------------")
#     print('\t ---------- EOL ----------')
#     print("\n\n\n")

questions = extract_question_block(lines)
print("QUESTION BLOCKS FIND")
print(len(questions))
print("\n\n\n\n\n")

print(questions[1])

for question in questions[1::]:

    unique_question = []
    for line in question:
        if line in unique_question:
            continue
        unique_question.append(line)
    # print(len(unique_question))
    # print(len(question))
    # print(unique_question)
    
    question_line = unique_question[0]['text']
    print(f'question : {question_line}')
    question_no = int(question_line.split(' ')[1])
    print(f'question_no : {question_no}')

    line_idx = 1
    note = ""
    # while line does not start with 'Question_no.1' it's teacher's comment
    while not unique_question[line_idx]['text'].lower().startswith(f'{question_no}.1'):
        note += unique_question[line_idx]['text'] + ' '
        line_idx += 1

    print(f'note : {note}')
    sub_question_no = 1
    sub_questions = []
    while line_idx < len(unique_question):
        answer = ""
        
        answer += unique_question[line_idx]['text']
        line_idx += 1

        while line_idx < len(unique_question) and not unique_question[line_idx]['text'].startswith(f'{question_no}.{sub_question_no + 1}'):
            answer += unique_question[line_idx]['text']
            line_idx += 1


        # ignore sub-question number (e.g. 1.1, 9.3, 20.2)
        if answer.startswith(f'{question_no}.{sub_question_no}'):
            answer = answer[len(f'{question_no}.{sub_question_no}')::]
        
        # while next work is not TRUE or FALSE, it's the question
        sub_question = ""
        word_idx = 0
        words = answer.split(' ')

        while word_idx < len(words) and not (words[word_idx].lower().startswith('true') or words[word_idx].lower().startswith('false')):
            sub_question += words[word_idx] + ' '
            word_idx += 1
        print(f'\tsub_question : {sub_question}')
        
        # extract true/false answer
        answer = words[word_idx]
        print(f'\tanswer : {answer}')
        
        # the rest is the explanation
        explanation = ' '.join(word for word in words[word_idx + 1::] if word != '')
        print(f'\texplanation : {explanation}')
        print("\n")

        
        
        sub_questions.append((sub_question, answer, explanation))
        sub_question_no += 1
        # print(f'\tanswer : {answer}')


    

print("\n\n\n\n\n")
    

# for page in pdf[1::]:
#     page = crop_header_footer_pdf_page(page)

#     for block in page.get_text('dict', flags=11)['blocks']:

#         for line in block['lines']:
#             for span in line['spans']:
#                 print(f"Font: {span['font']}\nText: {span['text']}\nSize: {span['size']}\nBbox: {span['bbox']}\n-------------------")
#             print('\t ---------- EOL ----------')
#         print('\t ---------- EOBlock ----------')
    
#     print('\t ---------- EOPage ----------')



# pdf.save('../data/EQE/2013_PreEx_answers_EN_cropped.pdf')


PRETRAITEMENT
QUESTION BLOCKS FIND
21






[{'size': 12.0, 'flags': 16, 'bidi': 0, 'char_flags': 16, 'font': 'Arial-BoldMT', 'color': 10, 'alpha': 255, 'ascender': 1.0180000066757202, 'descender': -0.37599998712539673, 'text': 'Question 1 ', 'origin': (71.99996948242188, 177.48486328125), 'bbox': (71.99996948242188, 165.26885986328125, 137.3759765625, 181.99685668945312)}, {'size': 12.0, 'flags': 0, 'bidi': 0, 'char_flags': 16, 'font': 'ArialMT', 'color': 10, 'alpha': 255, 'ascender': 1.00600004196167, 'descender': -0.32499998807907104, 'text': ' ', 'origin': (71.9999771118164, 198.244873046875), 'bbox': (71.9999771118164, 186.17286682128906, 75.33597564697266, 202.14486694335938)}, {'size': 12.0, 'flags': 0, 'bidi': 0, 'char_flags': 16, 'font': 'ArialMT', 'color': 10, 'alpha': 255, 'ascender': 1.00600004196167, 'descender': -0.32499998807907104, 'text': 'EP-G was filed within 12 months from the filing of IT-G, Article 87(1) EPC. Only ', 'origin': (71.9999771118164, 218.8848876953125)